<a href="https://colab.research.google.com/github/Followb1ind1y/D2L_Pytorch_Study_Notes/blob/main/07_D2L_Convolutional_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dive into Deep Learning 中文学习笔记** 
# **7. 卷积神经网络 （Convolutional Neural Networks）**

图像数据被表示为像素的二维网格，无论是单色的还是彩色的。因此，每个像素分别对应一个或多个数字值。到目前为止，我们忽略了这种丰富的结构，并通过 **扁平化（*flattening*）** 图像将它们作为数字的向量来处理，而不考虑像素之间的空间关系。从直觉上来说，我们认为相邻的像素通常是相互关联的。**卷积神经网络（*convolutional neural networks (CNNs)*）** 就是通过这样建立高效的模型，从图像数据中学习。


## **7.2. 用于图像的卷积层（Convolutions for Images）**

**卷积神经网络（*convolutional neural network*）** 是含有 **卷积层（*convolutional layer*）**的神经网络。

###**7.2.1. 互相关操作（The Cross-Correlation Operation）**

虽然卷积层得名于 **卷积（*convolution*）** 运算，但我们通常在卷积层中使用更加直观的 **互相关（*cross-correlation*）** 运算。在二维卷积层中，一个二维输入数组和一个二维 **核（*kernel*）** 数组通过互相关运算输出一个二维数组。如下图所示，输入是一个高和宽均为 3 的二维数组。我们将该数组的形状记为 $3\times 3$ 或（3，3。核数组的高和宽分别为 2。该数组在卷积计算中又称 **卷积核** 或 **过滤器（*filter*）**。卷积核窗口（又称卷积窗口）的形状取决于卷积核的高和宽，即 $2 \times 2$。图中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0×0+1×1+3×2+4×3=19$。

<center><img src="https://drive.google.com/uc?export=view&id=1foL6NZt-l-BAjszbfG06_Fnu2Wj-MSZH" width=350px /></center>

在二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。图中的输出数组高和宽分别为 2，其中的 4 个元素由二维互相关运算得出：

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.
$$



In [22]:
import torch
from torch import nn

def corr2d(X, K): 
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [23]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### **7.2.2. 卷积层（Convolutional Layers）**

二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。

下面基于 `corr2d` 函数来实现一个自定义的二维卷积层。在构造函数 `__init__` 里我们声明`weight` 和 `bias` 这两个模型参数。前向计算函数 `forward` 则是直接调用 `corr2d` 函数再加上偏差。

In [24]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

卷积窗口形状为 $p×q$ 的卷积层称为 $p×q$ 卷积层。同样，$p×q$ 卷积或 $p×q$ 卷积核说明卷积核的高和宽分别为 $p$ 和 $q$。

### **7.2.3. 图像中物体边缘检测（Object Edge Detection in Images）**

下面我们来看一个卷积层的简单应用：检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张 $6×8$ 的图像（即高和宽分别为 6 像素和 8 像素的图像）。它中间 4 列为黑（0），其余为白（1）。

In [25]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

然后我们构造一个高和宽分别为 1 和 2 的卷积核 $K$。当它与输入做互相关运算时，如果横向相邻元素相同，输出为 0；否则输出为非 0。

In [26]:
K = torch.tensor([[1.0, -1.0]])

下面将输入 $X$ 和我们设计的卷积核 $K$ 做互相关运算。可以看出，我们将从白到黑的边缘和从黑到白的边缘分别检测成了 1 和 -1。其余部分的输出全是 0。

In [27]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

由此，我们可以看出，卷积层可通过重复使用卷积核有效地表征局部空间。

我们现在可以将内核应用于 **转置（*transposed*）**后的图像。正如预期的那样，它消失了。内核 $K$ 只检测垂直边缘。

In [28]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### **7.2.4. 通过数据学习核数组（Learning a Kernel）**

最后我们来看一个例子，它使用物体边缘检测中的输入数据 $X$ 和输出数据 $Y$ 来学习我们构造的核数组 $K$。我们首先构造一个卷积层，其卷积核将被初始化成随机数组。接下来在每一次迭代中，我们使用平方误差来比较 $Y$ 和卷积层的输出，然后计算梯度来更新权重。

In [29]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 3.543
epoch 4, loss 0.881
epoch 6, loss 0.265
epoch 8, loss 0.093
epoch 10, loss 0.035


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


可以看到，10 次迭代后误差已经降到了一个比较小的值。现在来看一下学习到的卷积核的参数:

In [30]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9731, -1.0109]])

可以看到，学到的卷积核的权重参数与我们之前定义的核数组 $K$ 较接近，而偏置参数接近 0。

### **7.2.5. 互相关运算和卷积运算（Cross-Correlation and Convolution）**

实际上，卷积运算与互相关运算类似。**为了得到卷积运算的输出，我们只需将核数组左右翻转并上下翻转，再与输入数组做互相关运算**。可见，卷积运算和互相关运算虽然类似，但如果它们使用相同的核数组，对于同一个输入，输出往往并不相同。

那么，你也许会好奇卷积层为何能使用互相关运算替代卷积运算。其实，在深度学习中核数组都是学出来的：**卷积层无论使用互相关运算或卷积运算都不影响模型预测时的输出**。

### **7.2.6. 特征图和感受野（Feature Map and Receptive Field）**

二维卷积层输出的二维数组可以看作是输入在空间维度（宽和高）上某一级的表征，也叫 **特征图（*feature map*）**。影响元素 $x$ 的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做 $x$ 的 **感受野（*receptive field*）**。以下图为例，输入中阴影部分的四个元素是输出中阴影部分元素的感受野。我们将形状为 $2×2$ 的输出记为 $Y$，并考虑一个更深的卷积神经网络：将 $Y$ 与另一个形状为 $2×2$ 的核数组做互相关运算，输出单个元素 $z$。那么，$z$ 在 $Y$ 上的感受野包括 $Y$ 的全部四个元素，在输入上的感受野包括其中全部 9 个元素。可见，我们可以通过更深的卷积神经网络使特征图中单个元素的 **感受野变得更加广阔**，从而捕捉输入上更大尺寸的特征。

<center><img src="https://drive.google.com/uc?export=view&id=1foL6NZt-l-BAjszbfG06_Fnu2Wj-MSZH" width=350px /></center>

## **7.3. 填充和步幅（Padding and Stride）**

在上一节的例子里，我们使用高和宽为3的输入与高和宽为2的卷积核得到高和宽为2的输出。一般来说，假设输入形状是 $n_{h}×n_{w}$，卷积核窗口形状是 $k_{h}×k_{w}$，那么输出形状将会是: $(n_{h}-k_{h}+1)×(n_{w}-k_{w}+1)$。所以卷积层的输出形状由输入形状和卷积核窗口形状决定。

### **7.3.1. 填充（Padding）**

在应用卷积层时，我们往往会在图像的边缘丢失像素。像素的利用率可以被看作是卷积核大小和图像内位置的函数。而角落里的像素几乎没有被使用。由于我们通常使用小的核（kernels），对于任何给定的卷积，我们可能只损失几个像素，但当我们应用许多连续的卷积层时，这可能会增加。解决这个问题的一个直接办法是在输入图像的边界周围添加额外的填充像素，从而增加图像的有效尺寸。**填充（*padding*）** 是指在输入高和宽的两侧填充元素（通常是0元素）。

下图我们在原输入高和宽的两侧分别添加了值为 0 的元素，使得输入高和宽从 3 变成了 5，并导致输出高和宽由 2 增加到 4。图中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0×0+0×1+0×2+0×3=0$。

<center><img src="https://drive.google.com/uc?export=view&id=1oFChM0zzIXecuSm758PbsBXuI5wvLKUq" width=450px /></center>

一般来说，如果在高的两侧一共填充 $p_{h}$ 行，在宽的两侧一共填充 $p_{w}$ 列，那么输出形状将会是 

$$(n_{h}−k_{h}+p_{h}+1)×(n_{w}−k_{w}+p_{w}+1)$$ 

也就是说，输出的高和宽会分别增加$p_{h}$ 和 $p_{w}$。

在很多情况下，我们会设置 $p_{h}=k_{h}−1$ 和 $p_{w}=k_{w}−1$ 来使输入和输出具有相同的高和宽。这样会方便在构造网络时推测每个层的输出形状。假设这里 $k_{h}$ 是奇数，我们会在高的两侧分别填充 $p_{h}/2$ 行。如果 $k_{h}$ 是偶数，一种可能是在输入的顶端一侧填充 $⌈p_{h}/2⌉$ 行，而在底端一侧填充 $⌊p_{h}/2⌋$ 行。在宽的两侧填充同理。

卷积神经网络经常使用奇数高宽的卷积核，如1、3、5和7，所以两端上的填充个数相等。对任意的二维数组 $X$，设它的第 $i$ 行第 $j$ 列的元素为 $X[i,j]$。当两端上的填充个数相等，并使输入和输出具有相同的高和宽时，我们就知道输出 $Y[i,j]$ 是由输入以 $X[i,j]$ 为中心的窗口同卷积核进行互相关计算得到的。

下面的例子里我们创建一个高和宽为 3 的二维卷积层，然后设输入高和宽两侧的填充数分别为 1。给定一个高和宽为 8 的输入，我们发现输出的高和宽也是 8。

In [1]:
import torch
from torch import nn

# We define a helper function to calculate convolutions. It initializes the
# convolutional layer weights and performs corresponding dimensionality
# elevations and reductions on the input and output
def comp_conv2d(conv2d, X):
    # (1, 1) indicates that batch size and the number of channels are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Strip the first two dimensions: examples and channels
    return Y.reshape(Y.shape[2:])

# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([8, 8])

In [2]:
# We use a convolution kernel with height 5 and width 3. The padding on either
# side of the height and width are 2 and 1, respectively
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### **7.3.2. 步幅（Stride）**

在之前提到的二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。我们将每次滑动的行数和列数称为 **步幅（*stride*）**。跨步卷积往往能够帮助大幅降低维数。

目前我们看到的例子里，在高和宽两个方向上步幅均为 1。我们也可以使用更大步幅。下图展示了在高上步幅为 3、在宽上步幅为 2的二维互相关运算。可以看到，输出第一列第二个元素时，卷积窗口向下滑动了 3 行，而在输出第一行第二个元素时卷积窗口向右滑动了 2 列。当卷积窗口在输入上再向右滑动 2 列时，由于输入元素无法填满窗口，无结果输出。图中的阴影部分为输出元素及其计算所使用的输入和核数组元素：$0×0+0×1+1×2+2×3=8$、$0×0+6×1+0×2+0×3=6$。

<center><img src="https://drive.google.com/uc?export=view&id=1dOX8frWmGoJZivYrxr8FDFyGc7Yx47fg" width=450px /></center>

一般来说，当高上步幅为 $s_{h}$，宽上步幅为 $s_{w}$ 时，输出形状为:

$$
⌊(n_{h}−k_{h}+p_{h}+s_{h})/s_{h}⌋×⌊(n_{w}−k_{w}+p_{w}+s_{w})/s_{w}⌋
$$

如果设置 $p_{h}=k_{h}−1$ 和 $p_{w}=k_{w}−1$，那么输出形状将简化为 $⌊(n_{h}+s_{h}−1)/s_{h}⌋×⌊(n_{w}+s_{w}−1)/s_{w}⌋$。更进一步，如果输入的高和宽能分别被高和宽上的步幅整除，那么输出形状将是 $(n_{h}/s_{h})×(n_{w}/s_{w})$。

下面我们令高和宽上的步幅均为2，从而使输入的高和宽减半。

In [3]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

接下来是一个稍微复杂点儿的例子。

In [4]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

## **7.4. 多输入通道和多输出通道（Multiple Input and Multiple Output Channels）**

此前我们用到的输入和输出都是二维数组，但真实数据的维度经常更高。例如，彩色图像在高和宽 2 个维度外还有 RGB（红、绿、蓝）3 个颜色通道。假设彩色图像的高和宽分别是 $h$ 和 $w$（像素），那么它可以表示为一个 $3×h×w$ 的多维数组。我们将大小为 3 的这一维称为 **通道（*channel*）**。

### **7.4.1. 多输入通道（Multiple Input Channels）**

当输入数据含多个通道时，我们需要构造一个输入通道数与输入数据的通道数相同的卷积核，从而能够与含多通道的输入数据做互相关运算。假设输入数据的通道数为 $c_{i}$，那么卷积核的输入通道数同样为 $c_{i}$。设卷积核窗口形状为 $k_{h}×k_{w}$。当 $c_{i}=1$ 时，我们知道卷积核只包含一个形状为 $k_{h}×k_{w}$ 的二维数组。当 $c_{i}>1$ 时，我们将会为每个输入通道各分配一个形状为 $k_{h}×k_{w}$ 的核数组。把这 $c_{i}$ 个数组在输入通道维上连结，即得到一个形状为 $c_{i}×k_{h}×k_{w}$ 的卷积核。由于输入和卷积核各有 $c_{i}$ 个通道，我们可以在各个通道上对输入的二维数组和卷积核的二维核数组做互相关运算，再将这 $c_{i}$ 个互相关运算的二维输出按通道相加，得到一个二维数组。这就是含多个通道的输入数据与多输入通道的卷积核做二维互相关运算的输出。

下图展示了含 2 个输入通道的二维互相关计算的例子。在每个通道上，二维输入数组与二维核数组做互相关运算，再按通道相加即得到输出。图中阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：

$$(1×1+2×2+4×3+5×4)+(0×0+1×1+3×2+4×3)=56$$

<center><img src="https://drive.google.com/uc?export=view&id=1YZE642UQK85VCvazhq7IffVbndBkpSj-" width=550px /></center>


接下来我们实现含多个输入通道的互相关运算。我们只需要对每个通道做互相关运算，然后将结果进行累加。

In [ ]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [7]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### **7.4.2. 多输出通道（Multiple Output Channels）**

当输入通道有多个时，因为我们对各个通道的结果做了累加，所以不论输入通道数是多少，输出通道数总是为 1。设卷积核输入通道数和输出通道数分别为 $c_{i}$ 和 $c_{o}$，高和宽分别为 $k_{h}$ 和 $k_{w}$。如果希望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为 $c_{i}×k_{h}×k_{w}$ 的核数组。将它们在输出通道维上连结，卷积核的形状即 $c_{o}×c_{i}×k_{h}×k_{w}$。在做互相关运算时，每个输出通道上的结果由卷积核在该输出通道上的核数组与整个输入数组计算而来。

下面我们实现一个互相关运算函数来计算多个通道的输出：

In [8]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

我们将核数组 $K$ 同 $K+1$（$K$ 中每个元素加一）和 $K+2$ 连结在一起来构造一个输出通道数为 3 的卷积核。

In [9]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

下面我们对输入数组 $X$ 与核数组 $K$ 做互相关运算。此时的输出含有 3 个通道。其中第一个通道的结果与之前输入数组 $X$ 与多输入通道、单输出通道核的计算结果一致。

In [10]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### **7.4.3. $1×1$ 卷积层（$1\times 1$ Convolutional Layer）**

最后我们讨论卷积窗口形状为 $1×1$ $（kh=kw=1）$ 的多通道卷积层。我们通常称之为 $1×1$ 卷积层，并将其中的卷积运算称为 $1×1$ 卷积。因为使用了最小窗口，$1×1$ 卷积失去了卷积层可以识别高和宽维度上相邻元素构成的模式的功能。实际上，**$1×1$ 卷积的主要计算发生在通道维上**。下图展示了使用输入通道数为 3、输出通道数为 2 的 $1×1$ 卷积核的互相关计算。值得注意的是，输入和输出具有相同的高和宽。输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，**那么 $1×1$ 卷积层的作用与全连接层等价**。

<center><img src="https://drive.google.com/uc?export=view&id=17c-T385W7S04Pf2Z5z9FyKFzoKnczCw2" width=450px /></center>


下面我们使用全连接层中的矩阵乘法来实现 $1×1$ 卷积。这里需要在矩阵乘法运算前后对数据形状做一些调整:

In [11]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

经验证，做 $1×1$ 卷积时，以上函数与之前实现的互相关运算函数 `corr2d_multi_in_out` 等价。

In [12]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

在之后的模型里我们将会看到 $1×1$ 卷积层被当作保持高和宽维度形状不变的全连接层使用。于是，我们可以通过调整网络层之间的通道数来控制模型复杂度。

## **7.5. 池化层（Pooling）**

在之前二维卷积层中介绍的图像物体边缘检测应用中，我们构造卷积核从而精确地找到了像素变化的位置。设任意二维数组 $X$ 的 $i$ 行 $j$ 列的元素为 $X[i, j]$。如果我们构造的卷积核输出 $Y[i, j]=1$，那么说明输入中 $X[i, j]$ 和 $X[i, j+1]$ 数值不一样。这可能意味着物体边缘通过这两个元素之间。但实际图像里，我们感兴趣的物体不会总出现在固定位置：即使我们连续拍摄同一个物体也极有可能出现像素位置上的偏移。这会导致同一个边缘对应的输出可能出现在卷积输出 $Y$ 中的不同位置，进而对后面的模式识别造成不便。**池化（*pooling*）**层就提出是为了 **缓解卷积层对位置的过度敏感性**。

### **7.5.1. 最大池化层和平均池化层（Maximum Pooling and Average Pooling）**

同卷积层一样，池化层每次对输入数据的一个固定形状窗口（又称 **池化窗口（*pooling window*）**）中的元素计算输出。不同于卷积层里计算输入和核的互相关性，池化层直接计算池化窗口内元素的最大值或者平均值。该运算也分别叫做 **最大池化（*maximum pooling*）**或 **平均池化（*average pooling*）**。在二维最大池化中，池化窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当池化窗口滑动到某一位置时，窗口中的输入子数组的最大值即输出数组中相应位置的元素。

<center><img src="https://drive.google.com/uc?export=view&id=1tYzposaNzmsClIFcHUpcnsXMS3FB1fB6" width=350px /></center>

上图展示了池化窗口形状为 $2×2$ 的最大池化，阴影部分为第一个输出元素及其计算所使用的输入元素。输出数组的高和宽分别为 2，其中的 4 个元素由取最大值运算 `max` 得出：

$$
\max(0,1,3,4)=4 \\
\max(1,2,4,5)=5 \\
\max(3,4,6,7)=7 \\
\max(4,5,7,8)=8 \\
$$

平均池化的工作原理与最大池化类似，但将最大运算符替换成平均运算符。由于我们正在组合来自多个相邻像素的信息，因此我们可以对相邻像素进行平均以获得具有更好信噪比的图像。池化窗口形状为 $p×q$ 的池化层称为 $p×q$ 池化层，其中的池化运算叫作 $p×q$ 池化。

在物体边缘检测的例子中，我们现在将卷积层的输出作为 $2×2$ 最大池化的输入。设该卷积层输入是 $X$、池化层输出为 $Y$。无论是 $X[i, j]$ 和 $X[i, j+1]$ 值不同，还是 $X[i, j+1]$ 和 $X[i, j+2]$ 不同，池化层输出均有 $Y[i, j]=1$。也就是说，使用 $2×2$ 最大池化层时，只要卷积层识别的模式在高和宽上移动不超过一个元素，我们依然可以将它检测出来。

在下面的代码中，我们在函数中实现了池化层的前向传播 `pool2d`。该函数与 `corr2d `中的函数类似 。但是不需要内核，将输出计算为输入中每个区域的最大值或平均值。

In [13]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [14]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [15]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

### **7.5.2. 填充和步幅（Padding and Stride）**

同卷积层一样，池化层也可以在输入的高和宽两侧的填充并调整窗口的移动步幅来改变输出形状。池化层填充和步幅与卷积层填充和步幅的工作机制一样。我们将通过 `nn` 模块里的二维最大池化层 `MaxPool2d` 来演示池化层填充和步幅的工作机制。我们先构造一个形状为 `(1, 1, 4, 4)` 的输入数据，前两个维度分别是批量和通道。

In [16]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，`MaxPool2d` 实例里步幅和池化窗口形状相同。下面使用形状为 `(3, 3)` 的池化窗口，默认获得形状为 `(3, 3)` 的步幅。

In [17]:
pool2d = nn.MaxPool2d(3)
# Pooling has no model parameters, hence it needs no initialization
pool2d(X)

tensor([[[[10.]]]])

我们可以手动指定步幅和填充：

In [18]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

我们也可以指定非正方形的池化窗口，并分别指定高和宽上的填充和步幅：

In [19]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

### **7.5.3. 多通道（Multiple Channels）**

在处理多通道输入数据时，池化层对 **每个输入通道分别池化，而不是像卷积层那样将各通道的输入按通道相加**。这意味着池化层的输出通道数与输入通道数相等。下面将数组 $X$ 和 $X+1$ 在通道维上连结来构造通道数为 2 的输入。

In [20]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

池化后，我们发现输出通道数仍然是2:

In [21]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

In [32]:
%%shell
jupyter nbconvert --to html 07_D2L_Convolutional_Neural_Networks.ipynb

[NbConvertApp] Converting notebook 07_D2L_Convolutional_Neural_Networks.ipynb to html
[NbConvertApp] Writing 344881 bytes to 07_D2L_Convolutional_Neural_Networks.html
